In [308]:
import pubchempy as pcp
import pandas as pd

from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem.Draw import MolToImage

from PIL import ImageTk
import tkinter as tk

In [313]:
smiles

'C=C'

In [281]:
def obter_informacoes_molecula(mol):
    nome = mol
    compound = pcp.get_compounds(mol, 'name')
    smiles = compound[0].isomeric_smiles
    mol = Chem.MolFromSmiles(smiles)
    
    # Verificar se o SMILES foi convertido corretamente em uma molécula
    if mol is None:
        print(f"Erro ao processar o SMILES: {smiles}")
        return None
    
    informacoes = {
    "SMILES": smiles,
    "Nome": nome,
    "Fórmula Molecular": Chem.rdMolDescriptors.CalcMolFormula(mol),
    "Massa Molecular": Chem.rdMolDescriptors.CalcExactMolWt(mol),
    "Número de Átomos": mol.GetNumAtoms(),
    "Número de Ligações": mol.GetNumBonds(),
    "Área de Superfície": Chem.rdMolDescriptors.CalcTPSA(mol),
    "LogP": Chem.Crippen.MolLogP(mol),
    "Anéis Aromáticos": Chem.rdMolDescriptors.CalcNumAromaticRings(mol),
    "Número de Anéis": Chem.rdMolDescriptors.CalcNumRings(mol),
    "Número de Rotâmeros": Chem.rdMolDescriptors.CalcNumRotatableBonds(mol),
    "Polaridade": Chem.Descriptors.TPSA(mol),
    "Raio Molecular": Chem.Crippen.MolMR(mol)
    # Adicione outras informações desejadas
    }
    
    try:
        # Obter informações adicionais do PubChem
        composto = pcp.get_compounds(smiles, 'smiles')
        if composto:
            resultado = composto[0]
            
            if 'melting_point' in resultado.props:
                informacoes['Ponto de Fusão'] = resultado.props['melting_point']
            
            if 'boiling_point' in resultado.props:
                informacoes['Ponto de Ebulição'] = resultado.props['boiling_point']
    
    except Exception as e:
        print(f"Erro ao obter informações do PubChem: {e}")

    return informacoes

In [299]:
obter_informacoes_molecula('ethylene')
pka

Erro ao obter informações do PubChem: 'Compound' object has no attribute 'props'


{'SMILES': 'C=C',
 'Nome': 'ethylene',
 'Fórmula Molecular': 'C2H4',
 'Massa Molecular': 28.031300127999998,
 'Número de Átomos': 2,
 'Número de Ligações': 1,
 'Área de Superfície': 0.0,
 'LogP': 0.8022,
 'Anéis Aromáticos': 0,
 'Número de Anéis': 0,
 'Número de Rotâmeros': 0,
 'Polaridade': 0.0,
 'Raio Molecular': 11.254000000000001}

In [296]:
molecule = input("Molecule name: ")

try:
    df = pcp.get_compounds(molecule, 'name', as_dataframe=True)
    smiles = df["canonical_smiles"]
except:
    print('Unknown molecule name.')

results = pcp.get_compounds(molecule, 'name')
for compound in results:
    print(compound)
    mol = Chem.MolFromSmiles(compound.isomeric_smiles)
    print(Chem.rdMolDescriptors.CalcMolFormula(mol))

    # Criar uma janela popup
    window = tk.Tk()

    # Mostrar a imagem do benzeno em uma janela popup
    img = MolToImage(mol)
    photo = ImageTk.PhotoImage(img)
    label = tk.Label(window, image=photo)
    label.pack()
    window.title(f"{molecule}")
    window.mainloop()

Compound(18730)
C21H11NO5S


In [300]:
# Lista de grupos funcionais com seus SMARTS correspondentes
grupos_funcionais = {
    'Hidroxila': '[OH]',
    'Amina Primária': '[NH2]',
    'Amina Secundária': '[NH1;!$(NC=O)]',
    'Amina Terciária': '[NH0;!$(NC=O)]',
    'Carbonila Aldeído': '[C;H1](=O)',
    'Carbonila Cetona': '[C;H2](=O)',
    'Carbonila Ácido Carboxílico': '[C;H1](=O)[OH]',
    'Carbonila Éster': '[C;H1](=O)[O;!$(OC=O)]',
    'Carbonila Amida': '[C;H1](=O)[NH2]',
    'Enol': '[C;H1]([OH])(=C)',
    'Epóxido': '[O]1[C;H1][C;H1]1',
    'Diazonio': '[N;H0]#[N]',
    'Nitrila': '[N]#[C]',
    'Alqueno': '[C;H2]=[C;H2]',
    'Alquino': '[C;H]#C',
    'Halogeneto de Acila': '[C;H1](=O)[Cl,Br,I]',
    'Halogeneto de Alquila': '[Cl,Br,I]',
    'Grupo Nitro': '[N+](=O)[O-]',
    'Grupo Nitrosa': '[N+]([O-])[OH]',
    'Sulfonato': '[S;X4](=O)(=O)[O-]',
    'Grupo Carbonato': '[C]([O;X1])=[O;X1]',
    'Grupo Isocianato': '[N;X2]=C=O',
    'Grupo Azida': '[N-]=[N+]=[N-]',
    'Grupo Cianato': 'N=C=O',
    'Grupo Nitroso': '[N+]([O-])=O',
    'Grupo Alquil Grignard': '[Mg]([Br,Cl,I])([C])',
    'Grupo Acetil': '[CH3](=O)',
    'Grupo Carboxamida': '[C;H1](=O)[NH2]',
    'Grupo Sulfonamida': '[S;X4](=O)(=O)[NH2]',
    'Grupo Isotiocianato': '[N;X2]=C=S',
    'Grupo Peróxido': '[O]O',
    'Grupo Halogeneto de Alquila': '[F,Cl,Br,I]',
    'Grupo Hidrazida': 'NN',
    'Grupo Nitroso Amida': '[N+]([O-])[NH2]',
    'Grupo Nitroso Éster': '[N+]([O-])[O;!$(OC=O)]',
    'Grupo Isonitrila': '[N;X2]=C#[C]',
    'Grupo Sulfonil Clorido': 'S(=O)(=O)Cl',
    'Grupo Tioéter': '[S;X2][C;H3]',
}



compound = pcp.get_compounds("ethylene", 'name')

mol = Chem.MolFromSmiles(compound[0].isomeric_smiles)  # Exemplo de uma molécula (acetato)

# Converter a molécula para SMILES
smiles = Chem.MolToSmiles(mol)

# Identificar os grupos funcionais presentes na molécula
grupos_identificados = []
for nome, smarts in grupos_funcionais.items():
    grupo_mol = Chem.MolFromSmarts(smarts)
    matches = mol.GetSubstructMatches(grupo_mol)
    if len(matches) > 0:
        grupos_identificados.append(nome)

# Imprimir os grupos funcionais identificados
print("Grupos funcionais identificados na molécula:")
for grupo in grupos_identificados:
    print(grupo)


Grupos funcionais identificados na molécula:
Alqueno


In [ ]:
print(groups)

((2,),)


In [ ]:
print(Chem.rdMolDescriptors.CalcMolFormula(mol))